# LoRA 모델 성능 비교 분석

이 노트북은 베이스 모델과 LoRA 가중치를 로딩한 모델의 성능을 비교하여 시각적으로 확인할 수 있습니다.

## 기능
- 베이스 모델 (SD/SDXL) 이미지 생성
- LoRA 가중치를 로딩한 모델 이미지 생성
- 두 모델의 결과를 나란히 비교
- 다양한 프롬프트로 성능 테스트
- 파라미터 조정 실험

# 1. 준비 작업

In [1]:
# 필요한 라이브러리 import
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from enum import Enum
from typing import Any, Dict, Final, List, Tuple
from diffusers import StableDiffusionPipeline, DiffusionPipeline, EulerDiscreteScheduler
from diffusers.models import AutoencoderKL
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (선택사항)
plt.rcParams['font.family'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

/home/ubuntu/miniconda/envs/m_train/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda/envs/m_train/lib/python3.10/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 802: system not yet initialized (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


In [2]:
# 모델 ID 정의 (test_lora_local.py와 동일)
class HfModelId(str, Enum):
    # SD 모델
    SD_V1_5: str = "SG161222/Realistic_Vision_V5.1_noVAE"
    SD_VAE: str = "stabilityai/sd-vae-ft-mse"
    
    # SDXL 모델
    SDXL_V1_0_BASE: str = "stabilityai/stable-diffusion-xl-base-1.0"
    SDXL_V1_0_REFINER: str = "stabilityai/stable-diffusion-xl-refiner-1.0"

ENABLE_MODEL_CPU_OFFLOAD: Final = True
USE_REFINER: Final = False

In [3]:
# SD 모델 로딩 함수들
def load_sd_base_model() -> Any:
    """SD 베이스 모델 로딩 (LoRA 없이)"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading SD base model on device: {device}")

    model = StableDiffusionPipeline.from_pretrained(
        HfModelId.SD_V1_5.value, torch_dtype=torch.float16
    ).to(device)

    model.vae = AutoencoderKL.from_pretrained(
        HfModelId.SD_VAE.value, torch_dtype=torch.float16
    ).to(device)

    model.scheduler = EulerDiscreteScheduler.from_config(
        model.scheduler.config, use_karras_sigmas=True
    )

    return model

def load_sd_lora_model(model_dir: str) -> Any:
    """SD LoRA 모델 로딩 (베이스 + LoRA)"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading SD LoRA model on device: {device}")

    model = StableDiffusionPipeline.from_pretrained(
        HfModelId.SD_V1_5.value, torch_dtype=torch.float16
    ).to(device)

    model.vae = AutoencoderKL.from_pretrained(
        HfModelId.SD_VAE.value, torch_dtype=torch.float16
    ).to(device)

    model.scheduler = EulerDiscreteScheduler.from_config(
        model.scheduler.config, use_karras_sigmas=True
    )

    print(f"Loading LoRA weights from: {model_dir}")
    model.load_lora_weights(model_dir)

    return model

In [4]:
# SDXL 모델 로딩 함수들
def load_sdxl_base_model() -> Dict[str, Any]:
    """SDXL 베이스 모델 로딩 (LoRA 없이)"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading SDXL base model on device: {device}")

    model = DiffusionPipeline.from_pretrained(
        HfModelId.SDXL_V1_0_BASE.value,
        torch_dtype=torch.float16,
        variant="fp16",
    )
    
    _ = (
        model.enable_model_cpu_offload()
        if ENABLE_MODEL_CPU_OFFLOAD
        else model.to(device)
    )

    return {"model": model, "refiner": None}

def load_sdxl_lora_model(model_dir: str) -> Dict[str, Any]:
    """SDXL LoRA 모델 로딩 (베이스 + LoRA)"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading SDXL LoRA model on device: {device}")

    model = DiffusionPipeline.from_pretrained(
        HfModelId.SDXL_V1_0_BASE.value,
        torch_dtype=torch.float16,
        variant="fp16",
    )
    
    _ = (
        model.enable_model_cpu_offload()
        if ENABLE_MODEL_CPU_OFFLOAD
        else model.to(device)
    )

    print(f"Loading LoRA weights from: {model_dir}")
    model.load_lora_weights(model_dir)

    return {"model": model, "refiner": None}

In [5]:
# 이미지 생성 함수들
def generate_sd_image(model: Any, prompt: str, **kwargs) -> Image.Image:
    """SD 모델로 이미지 생성"""
    height = kwargs.get("height", 512)
    width = kwargs.get("width", 512)
    num_inference_steps = kwargs.get("num_inference_steps", 50)
    guidance_scale = kwargs.get("guidance_scale", 7.5)
    negative_prompt = kwargs.get("negative_prompt", None)
    num_images_per_prompt = kwargs.get("num_images_per_prompt", 1)
    seed = kwargs.get("seed", 42)
    cross_attention_scale = kwargs.get("cross_attention_scale", 0.5)

    negative_prompt = negative_prompt if negative_prompt and len(negative_prompt) > 0 else None
    device = "cuda" if torch.cuda.is_available() else "cpu"

    generator = torch.Generator(device=device).manual_seed(seed)
    generated_images = model(
        prompt,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_images_per_prompt,
        generator=generator,
        cross_attention_kwargs={"scale": cross_attention_scale},
    )["images"]

    return generated_images[0]

def generate_sdxl_image(model_dict: Dict[str, Any], prompt: str, **kwargs) -> Image.Image:
    """SDXL 모델로 이미지 생성"""
    height = kwargs.get("height", 1024)
    width = kwargs.get("width", 1024)
    num_inference_steps = kwargs.get("num_inference_steps", 50)
    guidance_scale = kwargs.get("guidance_scale", 7.5)
    negative_prompt = kwargs.get("negative_prompt", None)
    num_images_per_prompt = kwargs.get("num_images_per_prompt", 1)
    seed = kwargs.get("seed", 42)
    high_noise_frac = kwargs.get("high_noise_frac", 0.7)
    cross_attention_scale = kwargs.get("cross_attention_scale", 0.5)

    negative_prompt = negative_prompt if negative_prompt and len(negative_prompt) > 0 else None
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model, refiner = model_dict["model"], model_dict["refiner"]
    generator = torch.Generator(device=device).manual_seed(seed)

    if USE_REFINER and refiner:
        image = model(
            prompt=prompt,
            height=height,
            width=width,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_images_per_prompt,
            denoising_end=high_noise_frac,
            generator=generator,
            output_type="latent",
            cross_attention_kwargs={"scale": cross_attention_scale},
        )["images"]
        generated_images = refiner(
            prompt=prompt,
            image=image,
            num_inference_steps=num_inference_steps,
            denoising_start=high_noise_frac,
        )["images"]
    else:
        generated_images = model(
            prompt=prompt,
            height=height,
            width=width,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_images_per_prompt,
            generator=generator,
            cross_attention_kwargs={"scale": cross_attention_scale},
        )["images"]

    return generated_images[0]

In [6]:
# 이미지 비교 시각화 함수
def compare_images(base_image: Image.Image, lora_image: Image.Image, 
                  prompt: str, model_type: str = "SDXL", 
                  figsize: Tuple[int, int] = (12, 6)):
    """베이스 모델과 LoRA 모델의 이미지를 나란히 비교하여 표시"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    
    # 베이스 모델 이미지
    ax1.imshow(base_image)
    ax1.set_title(f'{model_type} Base Model\n{prompt[:50]}{"..." if len(prompt) > 50 else ""}', 
                  fontsize=12, fontweight='bold')
    ax1.axis('off')
    
    # LoRA 모델 이미지
    ax2.imshow(lora_image)
    ax2.set_title(f'{model_type} LoRA Model\n{prompt[:50]}{"..." if len(prompt) > 50 else ""}', 
                  fontsize=12, fontweight='bold')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 이미지 정보 출력
    print(f"\n=== 이미지 비교 결과 ===")
    print(f"프롬프트: {prompt}")
    print(f"베이스 모델 이미지 크기: {base_image.size}")
    print(f"LoRA 모델 이미지 크기: {lora_image.size}")
    print(f"모델 타입: {model_type}")

In [7]:
# 설정
MODEL_TYPE = "SDXL"  # "SD" 또는 "SDXL"
LORA_PATH = "../models/"  # LoRA 가중치 경로

print(f"모델 타입: {MODEL_TYPE}")
print(f"LoRA 경로: {LORA_PATH}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

모델 타입: SDXL
LoRA 경로: ../models/
CUDA 사용 가능: False


In [8]:
# 모델 로딩
print("=== 모델 로딩 시작 ===")

if MODEL_TYPE == "SD":
    # SD 모델 로딩
    print("\n1. SD 베이스 모델 로딩 중...")
    sd_base_model = load_sd_base_model()
    
    print("\n2. SD LoRA 모델 로딩 중...")
    sd_lora_model = load_sd_lora_model(LORA_PATH)
    
    base_model = sd_base_model
    lora_model = sd_lora_model
    
else:
    # SDXL 모델 로딩
    print("\n1. SDXL 베이스 모델 로딩 중...")
    sdxl_base_model = load_sdxl_base_model()
    
    print("\n2. SDXL LoRA 모델 로딩 중...")
    sdxl_lora_model = load_sdxl_lora_model(LORA_PATH)
    
    base_model = sdxl_base_model
    lora_model = sdxl_lora_model

print("\n=== 모델 로딩 완료 ===")

=== 모델 로딩 시작 ===

1. SDXL 베이스 모델 로딩 중...
Loading SDXL base model on device: cpu


Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.80it/s]



2. SDXL LoRA 모델 로딩 중...
Loading SDXL LoRA model on device: cpu


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.16it/s]


Loading LoRA weights from: ../models/

=== 모델 로딩 완료 ===


# 2. 실험 시작

In [9]:
# 테스트 프롬프트
test_prompts = [
    # 컨셉: 캐주얼 스트릿 스타일
    # 한국어: 오버사이즈 크림 후드티와 와이드 진으로 편안한 캐주얼 룩, 서울 쇼핑가 배경
    "Young Korean woman wearing oversized cream hoodie, wide-leg denim pants, chunky white sneakers, baseball cap, crossbody bag. Casual street style in Seoul shopping district.",
    
    # 컨셉: 미니멀 오피스 룩
    # 한국어: 테일러드 베이지 재킷과 스트레이트 팬츠로 완성한 미니멀 오피스 룩
    "Korean businesswoman in tailored beige blazer, white button shirt, black straight trousers, pointed-toe flats, structured handbag. Professional minimal aesthetic, office building background.",
    
    # 컨셉: 빈티지 레트로 스타일
    # 한국어: 70년대 빈티지 감성의 레트로 스타일, 플레어 진과 플랫폼 부츠 조합
    "Young Korean woman wearing vintage brown leather jacket, high-waisted flare jeans, platform boots, retro sunglasses, vintage band t-shirt. 70s inspired fashion, urban cafe setting.",
    
    # 컨셉: 로맨틱 페미닌 룩
    # 한국어: 미디 스커트와 핑크 가디건으로 완성한 로맨틱 페미닌 룩, 벚꽃 공원 배경
    "Korean woman in flowing midi skirt, soft pink cardigan, white blouse, mary-jane heels, pearl accessories. Romantic feminine style in cherry blossom park setting.",
    
    # 컨셉: 스포티 애슬레저
    # 한국어: 크롭 스포츠 재킷과 레깅스로 완성한 애슬레저 스타일
    "Young Korean woman wearing cropped sports jacket, high-waist leggings, chunky dad sneakers, baseball cap, fanny pack. Athletic streetwear style, urban gym exterior.",
    
    # 컨셉: 모던 한복 퓨전
    # 한국어: 현대적으로 재해석된 한복 퓨전 스타일, 크롭 저고리와 와이드 팬츠
    "Korean woman in modern hanbok-inspired outfit, cropped jeogori top, wide palazzo pants, traditional hair accessories, modern sneakers. Contemporary Korean fashion fusion style.",
    
    # 컨셉: 그런지 에지 스타일
    # 한국어: 찢어진 데님과 플래널 셔츠로 완성한 그런지 에지 스타일
    "Young Korean woman wearing distressed denim jacket, plaid flannel shirt, ripped black jeans, combat boots, chain accessories. Grunge aesthetic in underground music venue.",
    
    # 컨셉: 프레피 스쿨 룩
    # 한국어: 네이비 블레이저와 플리츠 스커트로 완성한 프레피 스쿨 룩
    "Korean student wearing navy blazer, white collar shirt, pleated skirt, knee-high socks, loafers, backpack. Preppy school uniform style, university campus background.",
    
    # 컨셉: 보헤미안 시크
    # 한국어: 맥시 드레스와 데님 재킷으로 완성한 보헤미안 시크 스타일
    "Young Korean woman in flowing maxi dress, denim jacket, ankle boots, layered jewelry, fringe bag, wide-brim hat. Bohemian chic style in outdoor music festival.",
    
    # 컨셉: 미니멀 모노톤
    # 한국어: 터틀넥과 울 코트로 완성한 모노톤 미니멀 스타일
    "Korean woman wearing black turtleneck, grey wool coat, black skinny pants, white minimalist sneakers, structured tote bag. Monochrome minimalist fashion, modern architecture background."
]

# 기본 파라미터
params = {
    "num_inference_steps": 30,  # 빠른 테스트를 위해 줄임
    "guidance_scale": 7.5,
    "seed": 42,
    "cross_attention_scale": 0.5
}

if MODEL_TYPE == "SD":
    params.update({"height": 512, "width": 512})
else:
    params.update({"height": 1024, "width": 1024})

print(f"테스트 파라미터: {params}")

테스트 파라미터: {'num_inference_steps': 30, 'guidance_scale': 7.5, 'seed': 42, 'cross_attention_scale': 0.5, 'height': 1024, 'width': 1024}


## 첫번째 프롬프트

In [10]:
# 첫 번째 프롬프트로 테스트
prompt = test_prompts[0]
print(f"\n=== 테스트 프롬프트: {prompt} ===")

# 베이스 모델로 이미지 생성
print("\n1. 베이스 모델로 이미지 생성 중...")
if MODEL_TYPE == "SD":
    base_image = generate_sd_image(base_model, prompt, **params)
else:
    base_image = generate_sdxl_image(base_model, prompt, **params)

# LoRA 모델로 이미지 생성
print("\n2. LoRA 모델로 이미지 생성 중...")
if MODEL_TYPE == "SD":
    lora_image = generate_sd_image(lora_model, prompt, **params)
else:
    lora_image = generate_sdxl_image(lora_model, prompt, **params)

# 결과 비교
compare_images(base_image, lora_image, prompt, MODEL_TYPE)


=== 테스트 프롬프트: Young Korean woman wearing oversized cream hoodie, wide-leg denim pants, chunky white sneakers, baseball cap, crossbody bag. Casual street style in Seoul shopping district. ===

1. 베이스 모델로 이미지 생성 중...


RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 802: system not yet initialized

In [ ]:
# 여러 프롬프트로 테스트
for i, prompt in enumerate(test_prompts[1:], 1):
    print(f"\n{'='*60}")
    print(f"테스트 {i+1}: {prompt}")
    print(f"{'='*60}")
    
    # 베이스 모델로 이미지 생성
    print("베이스 모델로 이미지 생성 중...")
    if MODEL_TYPE == "SD":
        base_image = generate_sd_image(base_model, prompt, **params)
    else:
        base_image = generate_sdxl_image(base_model, prompt, **params)

    # LoRA 모델로 이미지 생성
    print("LoRA 모델로 이미지 생성 중...")
    if MODEL_TYPE == "SD":
        lora_image = generate_sd_image(lora_model, prompt, **params)
    else:
        lora_image = generate_sdxl_image(lora_model, prompt, **params)

    # 결과 비교
    compare_images(base_image, lora_image, prompt, MODEL_TYPE)

## 사용자 정의 프롬프트
- Negatve 반영

In [12]:
# 사용자 정의 프롬프트 테스트 함수
def test_custom_prompt(prompt: str, negative_prompt: str = "", 
                      guidance_scale: float = 7.5, 
                      cross_attention_scale: float = 0.5):
    """사용자 정의 프롬프트로 베이스 모델과 LoRA 모델 비교"""
    
    print(f"\n=== 사용자 정의 프롬프트 테스트 ===")
    print(f"프롬프트: {prompt}")
    print(f"네거티브 프롬프트: {negative_prompt if negative_prompt else '(없음)'}")
    
    # 파라미터 설정
    test_params = params.copy()
    test_params.update({
        "guidance_scale": guidance_scale,
        "cross_attention_scale": cross_attention_scale,
        "negative_prompt": negative_prompt
    })
    
    # 베이스 모델로 이미지 생성
    print("\n1. 베이스 모델로 이미지 생성 중...")
    if MODEL_TYPE == "SD":
        base_image = generate_sd_image(base_model, prompt, **test_params)
    else:
        base_image = generate_sdxl_image(base_model, prompt, **test_params)

    # LoRA 모델로 이미지 생성
    print("\n2. LoRA 모델로 이미지 생성 중...")
    if MODEL_TYPE == "SD":
        lora_image = generate_sd_image(lora_model, prompt, **test_params)
    else:
        lora_image = generate_sdxl_image(lora_model, prompt, **test_params)

    # 결과 비교
    compare_images(base_image, lora_image, prompt, MODEL_TYPE)
    
    return base_image, lora_image

In [ ]:
# 예시: 사용자 정의 프롬프트 테스트
# 아래 프롬프트를 원하는 것으로 변경하여 테스트하세요

custom_prompt = "Young Korean woman with short brown bob hairstyle, wearing oversized gray crewneck sweatshirt with embroidered logo patch, distressed blue denim shorts with frayed hems, white knee-high socks with black stripes, chunky white sneakers with platform sole and colorful laces. Standing casually, hand touching hair. Youthful street style with comfortable oversized fit and vintage-inspired details."
custom_negative = "blurry, low quality, distorted"

base_img, lora_img = test_custom_prompt(
    prompt=custom_prompt,
    negative_prompt=custom_negative,
    guidance_scale=8.0,
    cross_attention_scale=0.6
)